In [1]:
import keras


Using TensorFlow backend.


In [2]:

from keras.applications import VGG16
#Load the VGG model
vgg_conv = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [3]:
# Freeze the layers except the last 4 layers
for layer in vgg_conv.layers[:-4]:
    layer.trainable = False
 
# Check the trainable status of the individual layers
for layer in vgg_conv.layers:
    print(layer, layer.trainable)

<keras.engine.input_layer.InputLayer object at 0xb324e22e8> False
<keras.layers.convolutional.Conv2D object at 0xb324e28d0> False
<keras.layers.convolutional.Conv2D object at 0xb324e2748> False
<keras.layers.pooling.MaxPooling2D object at 0xb3250d940> False
<keras.layers.convolutional.Conv2D object at 0xb3250ddd8> False
<keras.layers.convolutional.Conv2D object at 0xb3254e278> False
<keras.layers.pooling.MaxPooling2D object at 0xb325676d8> False
<keras.layers.convolutional.Conv2D object at 0xb32567550> False
<keras.layers.convolutional.Conv2D object at 0xb32594e80> False
<keras.layers.convolutional.Conv2D object at 0xb325c80b8> False
<keras.layers.pooling.MaxPooling2D object at 0xb325e16a0> False
<keras.layers.convolutional.Conv2D object at 0xb325e1518> False
<keras.layers.convolutional.Conv2D object at 0xb32615e48> False
<keras.layers.convolutional.Conv2D object at 0xb32631400> False
<keras.layers.pooling.MaxPooling2D object at 0xb3266f550> False
<keras.layers.convolutional.Conv2D obj

In [4]:
from keras import models
from keras import layers
from keras import optimizers
 
# Create the model
model = models.Sequential()
 
# Add the vgg convolutional base model
model.add(vgg_conv)
 
# Add new layers
model.add(layers.Flatten())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(95, activation='softmax'))
 
# Show a summary of the model. Check the number of trainable parameters
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 95)                97375     
Total params: 40,503,199
Trainable params: 32,867,935
Non-trainable params: 7,635,264
_________________________________________________________________


In [5]:
from keras.preprocessing.image import ImageDataGenerator
image_size=224
#train_dir = 'img/catsvsdogs'
#validation_dir = 'img/catsvsdogs'

train_dir = '/Users/gauravtyagi/Downloads/data/fruit/Training'
validation_dir = '/Users/gauravtyagi/Downloads/data/fruit/Test'


train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      width_shift_range=0.2,
      height_shift_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 100
val_batchsize = 10
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(image_size, image_size),
        batch_size=train_batchsize,
        class_mode='categorical')
 
validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(image_size, image_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)

Found 48905 images belonging to 95 classes.
Found 16421 images belonging to 95 classes.


In [ ]:

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])
# Train the model
history = model.fit_generator(
      train_generator,
      steps_per_epoch=train_generator.samples/train_generator.batch_size ,
      epochs=30,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,
      verbose=1)
 
# Save the model
model.save('/Users/gauravtyagi/Downloads/data/fruit/model')

Epoch 1/30
